In [1]:
import pandas as pd
from geopy import distance

This notebook helps calculate the closest pass for each hurricane to an affected county.  The 'hurricanes_data_v3.csv' was generated by modifying
the hurricane data notebook to filter for all observations of TD or greater strength as opposed to landfall at hurricane strength.  This relies on the 'final_merge.csv' generated by the fema_census_hur_merge notebook.

In [2]:
hur_data=pd.read_csv('hurricanes_data_v3.csv')

In [3]:
#convert date to string to slice it for year
hur_data['Date']=hur_data['Date'].astype('str')

In [4]:
#dropping the columns that will not be used in analysis
hur_data.drop(columns=['Unnamed: 0', 'Code', 'Time in UTC',
       'Record Identifier', 'Status',
       '34 kt wind radii maximum extent in northeastern quadrant',
       '34 kt wind radii maximum extent in southeastern quadrant',
       '34 kt wind radii maximum extent in northwestern quadrant',
       '34 kt wind radii maximum extent in southwestern quadrant',
       '50 kt wind radii maximum extent in northeastern quadrant',
       '50 kt wind radii maximum extent in southeastern quadrant',
       '50 kt wind radii maximum extent in northwestern quadrant',
       '50 kt wind radii maximum extent in southwestern quadrant',
       '64 kt wind radii maximum extent in northeastern quadrant',
       '64 kt wind radii maximum extent in southeastern quadrant',
       '64 kt wind radii maximum extent in northwestern quadrant',
       '64 kt wind radii maximum extent in southwestern quadrant',
       'Radius of Max Wind'],inplace=True)

In [5]:
#obtain the year for merging.  This is important since some hurricane names are used twice in the dataset
hur_data['year']=hur_data['Date'].str.slice(start=0,stop=4)

In [6]:
hur_data['year']=hur_data['year'].astype('int64')

In [7]:
#the HURDAT2 dataset records lat/long as strings such as 64.5W which would be converted to '-64.4' to merge with FEMA data
#and to support calculations in geopandas
hur_data['new_Lat']=hur_data['Latitude'].str[:-1]
hur_data['new_Lon']='-'+hur_data['Longitude'].str.lstrip().str[:-1]

In [8]:
hur_data.drop(columns=['Latitude','Longitude'],inplace=True)

In [9]:
#filtering only for hurricanes in timeframe of interest
hur_data=hur_data[hur_data['year']>=2003]

In [10]:
#uploading the dataset that contains all features for analysis.  This was the original 'final' dataset that needed modification
#after discovering our supervised model had low predictive power.
final_merge=pd.read_csv('final_merge.csv')

In [11]:
#storm list to ensure that we obtain information only for storms we have complete informaton for
storm_list=final_merge['Name'].unique().tolist()

In [12]:
#initialize empty dataframe
closest_pass_df=pd.DataFrame(columns=['Name','year','Max Wind','Min Pressure','fipsCode','distance_from_storm','category'])

In [13]:
#nested loops to extract information for each storm.  Since a few storm names are employed more than once, we have to 
#loop through storms and years simultaneously.
counter=0
for storm in storm_list:
    storm_df=hur_data[hur_data['Name']==storm]
    year_list=storm_df['year'].unique().tolist()
    
    for year in year_list:
        #create empty list of coordinates for all 6-hour hurricane positions.  
        #Create smaller dataframes for use in calculating distances with geopandas 
        hur_coords=[]
        storm_df2=storm_df[storm_df['year']==year]
        storm_df_counties=final_merge[(final_merge['Name']==storm)&(final_merge['year']==year)]

        #get coordinates for the storms
        for j in range(len(storm_df2)):
            hur_coords.append((storm_df2['new_Lat'].iloc[j],storm_df2['new_Lon'].iloc[j]))

        #get coordinates for impacted counties individually for geopandas analysis
        for i in range(len(storm_df_counties)):
            county_coord=(storm_df_counties['LATITUDE_county'].iloc[i],storm_df_counties['LONGITUDE_county'].iloc[i])

            #initialize variables for closest pass observation and distance
            closest_k=0
            closest_distance=1000

            #calculate the closest distance between hurricane center and county center
            for k in range(len(hur_coords)):
                current_dist=distance.distance(county_coord,hur_coords[k]).miles
                if current_dist<closest_distance:
                    closest_distance=current_dist
                    closest_k=k

            #obtain wind, minimum pressure and fipsCode for each county at closest pass
            wind=storm_df2['Max Wind'].iloc[closest_k]
            pressure=storm_df2['Min Pressure'].iloc[closest_k]
            fips=storm_df_counties['fipsCode'].iloc[i]
            category=None

            #use wind to calculate storm category
            if wind>=157:
                category='5'
            elif (wind<=156)&(wind>=130):
                category='4'
            elif (wind<=129)&(wind>=111):
                category='3'
            elif (wind<=110)&(wind>=96):
                category='2'
            elif (wind<=95)&(wind>=74):
                category='1'
            elif (wind<=73)&(wind>=39):
                category='TS'
            elif (wind<=38):
                category='TD'

            #create a datframe with closest pass
            closest_pass=pd.DataFrame([[storm,year,wind,pressure,fips,closest_distance,category]],
                                         columns=['Name','year','Max Wind','Min Pressure','fipsCode','distance_from_storm','category'])

            #concatenate to combined closest pass dataframe
            closest_pass_df=pd.concat([closest_pass_df,closest_pass], ignore_index=True)
            
            counter+=1
#print counter to ensure all observations are kept
print(counter)

/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_38535/3131063822.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  closest_pass_df=pd.concat([closest_pass_df,closest_pass], ignore_index=True)


1730


In [14]:
closest_pass_df.to_csv('closest_pass.csv',index=False)

In [15]:
#read in cluster.csv and combine with final merge and closest_pass_df
cluster_df=pd.read_csv('clusters_n_2.csv')

In [16]:
closest_pass_df.rename(columns={'Max Wind':'Max Wind closest','Min Pressure':'Min Pressure closest',
                                'distance_from_storm':'closest_pass'},inplace=True)

In [17]:
#ensure observations have not been duplicated
cluster_df.drop_duplicates(subset=['fipsCode'],inplace=True)
len(cluster_df)

741

In [18]:
final_merge_1=final_merge.merge(closest_pass_df,how='inner',on=['Name','year','fipsCode'])

In [19]:
len(final_merge_1)

1730

In [20]:
#use left join since economic cluster does not apply to every county
final_merge_2=final_merge_1.merge(cluster_df, how='left',on=['fipsCode'])

In [21]:
len(final_merge_2)

1730

In [22]:
#create dataframe for analysis
analysis_df=final_merge_2[[
 'DamageBetween20001and3000',
 'DamageGreaterThan3000',
 'MajorDamage_rent',
 'Estimate!!HOUSING OCCUPANCY!!Total housing units',
 'Percent!!HOUSING OCCUPANCY!!Total housing units',
 'Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units',
 'Percent!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2020 or later',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2020 or later',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2010 to 2019',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2010 to 2019',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2000 to 2009',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2000 to 2009',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1990 to 1999',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1990 to 1999',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1980 to 1989',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1980 to 1989',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1970 to 1979',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1970 to 1979',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1960 to 1969',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1960 to 1969',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1950 to 1959',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1950 to 1959',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1940 to 1949',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1940 to 1949',
 'Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier',
 'Percent!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier',
 'LATITUDE_county', 'dist_from_landfall',
 'LONGITUDE_county',
 'Landfall_Lat',
 'Landfall_Lon',
 'Max Wind',
 'Min Pressure','Min Pressure closest','Max Wind closest','category','closest_pass', 'housing_cluster','econ_cluster']]

KeyError: "['Min Pressure'] not in index"

In [ ]:
#drop county latitude and longitude for center of population as well as landfall latitude and longitude
#these will not be needed for supervised learning.
analysis_df.drop(columns=['LATITUDE_county',
 'LONGITUDE_county',
 'Landfall_Lat',
 'Landfall_Lon',],inplace=True)

C:\Users\sbpow\AppData\Local\Temp\ipykernel_12144\2984265007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_df.drop(columns=['LATITUDE_county',


In [ ]:
len(analysis_df)

1730

In [ ]:
#new analysis df with clusters and closest pass information
analysis_df.to_csv('analysis_df_v2.csv',index=False)